# `-` 대회에서의 `Unet` 과 Unet3+ 에서 쓰는 함수를 이어서 사용하기

# 1.Imports
`-` 변수로써 작용할만한 중요한 것만
```
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
```

# 2. 대회 내에서 Unet 설정


`-` 요약
- ConvBlock(인풋채널,아웃풋채널,커널3) 받아서 conv -> 배치노말 -> relu 진행 
- DeconvBlock(인풋채널,미드채널,아웃풋채널) 받아서 Upsampling 및 conv -> 배치노말 -> relu -> skip connection -> convblock 진행
- Unet 모델 : train 에 넣어서 model() 돌리면 끝.


```python

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3):
        super(ConvBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, padding=1)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.relu(self.bn(self.conv(x)))

class DeconvBlock(nn.Module):
    def __init__(self, in_channels, mid_channels, out_channels):
        super(DeconvBlock, self).__init__()
        self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.conv_mid = nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1)
        self.bn_mid = nn.BatchNorm2d(mid_channels)
        self.relu_mid = nn.ReLU()
        self.conv_block = ConvBlock(mid_channels + mid_channels, out_channels)

    def forward(self, x, skip):
        x = self.up(x)
        x = self.relu_mid(self.bn_mid(self.conv_mid(x)))
        x = torch.cat((x, skip), dim=1)
        return self.conv_block(x)
class BaseModel(nn.Module):
    def __init__(self):
        super(BaseModel, self).__init__()
        # Contraction path
        self.conv1 = ConvBlock(3, 16)
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = ConvBlock(16, 32)
        self.pool2 = nn.MaxPool2d(2)
        self.conv3 = ConvBlock(32, 64)
        self.pool3 = nn.MaxPool2d(2)
        self.conv4 = ConvBlock(64, 128)
        self.pool4 = nn.MaxPool2d(2)
        self.conv5 = ConvBlock(128, 256)

        # Expansion path
        self.up6 = DeconvBlock(256, 128, 128)
        self.up7 = DeconvBlock(128, 64, 64)
        self.up8 = DeconvBlock(64, 32, 32)
        self.up9 = DeconvBlock(32, 16, 16)

        self.final_pool = nn.MaxPool2d(2)
        self.final_conv = nn.Conv2d(16, 16, kernel_size=28, stride=28)
        self.final_bn = nn.BatchNorm2d(16)

    def forward(self, x):
        # Contraction path
        x1 = self.conv1(x)
        x = self.pool1(x1)
        x2 = self.conv2(x)
        x = self.pool2(x2)
        x3 = self.conv3(x)
        x = self.pool3(x3)
        x4 = self.conv4(x)
        x = self.pool4(x4)
        x5 = self.conv5(x)

        # Expansion path
        x = self.up6(x5, x4)
        x = self.up7(x, x3)
        x = self.up8(x, x2)
        x = self.up9(x, x1)

        x = self.final_pool(x)
        out = self.final_bn(self.final_conv(x)) # (B,16,4,4)
        return out

   
 ```

`-` train 설정
```python
def train(model, optimizer, train_loader, val_loader, device):
    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    
    best_val_acc = 0
    best_model = None
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            output = model(imgs)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_acc = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val ACC : [{_val_acc:.5f}]')
        
        if best_val_acc < _val_acc:
            best_val_acc = _val_acc
            best_model = model
 ```

위에서 보니 중요한건 train_loader 에 무엇이 들어가느냐인것같습니다.

`-` train_loader 를 보자.
```python
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transform=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transform = transform
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]

        # PIL 이미지로 불러오기
        image = Image.open(img_path).convert("RGB")
        if self.transform is not None:
            image = self.transform(image)
        
        if self.label_list is not None:
            label = torch.tensor(self.label_list[index], dtype=torch.long) - 1
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((CFG['IMG_SIZE'], CFG['IMG_SIZE'])),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((CFG['IMG_SIZE'], CFG['IMG_SIZE'])),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
train_dataset = CustomDataset(train_df['img_path'].values, train_labels, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = CustomDataset(val_df['img_path'].values, val_labels, test_transform)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

test_dataset = CustomDataset(test_df['img_path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

```

## 대회내에서 train 을 돌리면
`1` 
```
model = BaseModel()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
infer_model = train(model, optimizer, train_loader, val_loader, device)
```
`1 - (1)` 현재 이곳에서 BaseModel() 은 그대로이고 input 과 output 이 중요한데 infer_model 에서 봐야한다. <br>
`1 - (2)` infer_model 은 train 함수이고 `train_loader` 이 무엇인지 볼 필요가 있다. <br>
`1 - (3)` train_loader 은 DataLoader 이란 pytorch 의 함수를 받고 train_dataset 이라는 CustomDataset 이 적용되어 만들어져있다. <br>
`1 - (4)` CustomDataset 에서는 보통 img,label 설정 및 __len__ , __getitem__ 등의 설정을 하는 곳이라 유의깊게 봐야한다. <br>
`1 - (결론)` train_dataset 은 img= 이미지RGB 로 들어가서 tensor 로 나온 tensor 값들이 들어가게된다.
`1 - (완결)` 그렇다면 train_loader 까지는 똑같을 것이고 train 함수자체와 BaseModel 이 우리가 새로 찾은 모델과 무엇이 다른지 알아보자.

# 3. Unet3+ 에서의 Unet 설정